In [ ]:
!python -V

In [ ]:
#@markdown **Step 1:** Check which GPU you've been allocated.

!nvidia-smi -L
!nvidia-smi

In [ ]:
#@markdown **Step 2:** Download dependencies.
%tensorflow_version 2.x
import os

custom_lists = [
    #"https://gist.githubusercontent.com/SortAnon/997cda157954a189259c9876fd804e53/raw/example_models.json",
]

!apt-get install sox libsndfile1 ffmpeg
!pip install tensorflow==2.4.1 dash==1.21.0 dash-bootstrap-components==0.13.0 jupyter-dash==0.4.0 psola wget unidecode pysptk frozendict torchvision==0.9.1 torchaudio==0.8.1 torchtext==0.9.1 torch_stft kaldiio pydub pyannote.audio g2p_en pesq pystoi crepe resampy ffmpeg-python torchcrepe einops taming-transformers-rom1504==0.0.6 tensorflow-hub gdown --upgrade
!python -m pip install git+https://github.com/SortAnon/NeMo.git
if not os.path.exists("hifi-gan"):
    !git clone -q --recursive https://github.com/SortAnon/hifi-gan
!git clone -q https://github.com/SortAnon/ControllableTalkNet
os.chdir("/content/ControllableTalkNet")
!git archive --output=./files.tar --format=tar HEAD
os.chdir("/content")
!tar xf ControllableTalkNet/files.tar
!rm -rf ControllableTalkNet

# PESQ fix
!python -m pip uninstall -y pesq
!python -m pip uninstall -y numpy
!python -m pip install numpy==1.19.5
!python -m pip --no-cache-dir install --no-build-isolation --no-binary :all: pesq==0.0.2

os.chdir("/content/model_lists")
for c in custom_lists:
    !wget "{c}"
os.chdir("/content")


#Download pretrained model

In [ ]:
!sudo wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=17vecivV8k2zEBmwAHLSFN-8UoLkJK5LQ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=17vecivV8k2zEBmwAHLSFN-8UoLkJK5LQ" -O Character_Talknet.zip && rm -rf /tmp/cookies.txt

In [ ]:
!unzip /content/Character_Talknet.zip

In [6]:
!chmod -R 777 /content/

#Define class and functions

In [1]:
import json
import os
import sys
import numpy as np
import resampy
import scipy
import torch

sys.path.append("hifi-gan")
from denoiser import Denoiser
from env import AttrDict
from meldataset import MAX_WAV_VALUE, mel_spectrogram
from models import Generator


class vHiFiGAN:
    def __init__(self, model_path, conf_name, device):
        # Load HiFi-GAN
        conf = os.path.join("hifi-gan", conf_name + ".json")
        with open(conf) as f:
            json_config = json.loads(f.read())
        self.h = AttrDict(json_config)
        torch.manual_seed(self.h.seed)
        self.hifigan = Generator(self.h).to(torch.device(device))
        state_dict_g = torch.load(model_path, map_location=torch.device(device))
        self.hifigan.load_state_dict(state_dict_g["generator"])
        self.hifigan.eval()
        self.hifigan.remove_weight_norm()
        self.denoiser = Denoiser(self.hifigan, mode="normal")
        self.device = device

    def vocode(self, spect):
        y_g_hat = self.hifigan(spect.float())
        audio = y_g_hat.squeeze()
        audio = audio * MAX_WAV_VALUE
        audio_denoised = self.denoiser(audio.view(1, -1), strength=35)[:, 0]
        return (
            audio_denoised.detach().cpu().numpy().reshape(-1).astype(np.int16),
            audio_denoised.detach().cpu(),
        )

    def superres(self, audio, original_sr):
        # Resampling
        wave = resampy.resample(
            audio,
            original_sr,
            self.h.sampling_rate,
            filter="sinc_window",
            window=scipy.signal.windows.hann,
            num_zeros=8,
        )
        wave_out = wave.astype(np.int16)

        # Super-res
        wave = wave / MAX_WAV_VALUE
        wave = torch.FloatTensor(wave).to(torch.device(self.device))
        new_mel = mel_spectrogram(
            wave.unsqueeze(0),
            self.h.n_fft,
            self.h.num_mels,
            self.h.sampling_rate,
            self.h.hop_size,
            self.h.win_size,
            self.h.fmin,
            self.h.fmax,
        )
        y_g_hat2 = self.hifigan(new_mel)
        audio2 = y_g_hat2.squeeze()
        audio2 = audio2 * MAX_WAV_VALUE
        audio2_denoised = self.denoiser(audio2.view(1, -1), strength=35)[:, 0]

        # High-pass filter, mixing and denormalizing
        audio2_denoised = audio2_denoised.detach().cpu().numpy().reshape(-1)
        b = scipy.signal.firwin(
            101, cutoff=10500, fs=self.h.sampling_rate, pass_zero=False
        )
        y = scipy.signal.lfilter(b, [1.0], audio2_denoised)
        y *= 4.0  # superres strength
        y_out = y.astype(np.int16)
        y_padded = np.zeros(wave_out.shape)
        y_padded[: y_out.shape[0]] = y_out
        sr_mix = wave_out + y_padded
        return sr_mix, self.h.sampling_rate


In [2]:
import torch
from nemo.collections.tts.models import TalkNetSpectModel, TalkNetDursModel, TalkNetPitchModel,HifiGanModel

spec_gen = TalkNetSpectModel.restore_from('TalkNetSpect.nemo')
spec_gen.add_module('_pitch_model', TalkNetPitchModel.restore_from('TalkNetPitch.nemo'))
spec_gen.add_module('_durs_model', TalkNetDursModel.restore_from('TalkNetDurs.nemo'))
vocoder = vHiFiGAN('hifiganmodel', "config_v1", "cuda:0")


[NeMo W 2022-03-29 12:57:22 optimizers:47] Apex was not found. Using the lamb optimizer will error out.
[NeMo W 2022-03-29 12:57:23 nemo_logging:349] c:\Users\stymlice\Documents\GitHub\labTalkNet\venv\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
      warnings.warn('torchaudio C++ extension is not available.')
    
[NeMo W 2022-03-29 12:57:25 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-03-29 12:57:26 modelPT:139] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.asr.data.audio_to_text.AudioToCharWithDursF0Dataset
      manifest_filepath: trainfiles.json
      max_duration: null


[NeMo I 2022-03-29 12:57:26 features:252] PADDING: 1
[NeMo I 2022-03-29 12:57:26 features:269] STFT using torch


[NeMo W 2022-03-29 12:57:26 nemo_logging:349] c:\Users\stymlice\Documents\GitHub\labTalkNet\venv\lib\site-packages\nemo\collections\asr\parts\preprocessing\features.py:302: FutureWarning: Pass sr=22050, n_fft=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
      librosa.filters.mel(sample_rate, self.n_fft, n_mels=nfilt, fmin=lowfreq, fmax=highfreq), dtype=torch.float
    


[NeMo I 2022-03-29 12:57:30 modelPT:439] Model TalkNetSpectModel was successfully restored from TalkNetSpect.nemo.


[NeMo W 2022-03-29 12:57:30 modelPT:139] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.asr.data.audio_to_text.AudioToCharWithDursF0Dataset
      manifest_filepath: trainfiles.json
      max_duration: null
      min_duration: 0.1
      int_values: false
      load_audio: false
      normalize: false
      sample_rate: 22050
      trim: false
      durs_file: /content/drive/My Drive/LPL/durations.pt
      f0_file: /content/drive/My Drive/LPL/f0s.pt
      blanking: true
      vocab:
        notation: phonemes
        punct: true
        spaces: true
        stresses: false
        add_blank_at: last
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 16
      num_workers: 4
    
[NeMo W 2022-03-29 12:57:30 modelPT:146] If you intend to do validation, please call the Model

[NeMo I 2022-03-29 12:57:30 modelPT:439] Model TalkNetPitchModel was successfully restored from TalkNetPitch.nemo.


[NeMo W 2022-03-29 12:57:31 modelPT:139] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.asr.data.audio_to_text.AudioToCharWithDursF0Dataset
      manifest_filepath: trainfiles.json
      max_duration: null
      min_duration: 0.1
      int_values: false
      load_audio: false
      normalize: false
      sample_rate: 22050
      trim: false
      durs_file: /content/drive/My Drive/LPL/durations.pt
      f0_file: /content/drive/My Drive/LPL/f0s.pt
      blanking: true
      vocab:
        notation: phonemes
        punct: true
        spaces: true
        stresses: false
        add_blank_at: last
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-03-29 12:57:31 modelPT:146] If you intend to do validation, please call the Model

[NeMo I 2022-03-29 12:57:31 modelPT:439] Model TalkNetDursModel was successfully restored from TalkNetDurs.nemo.
Removing weight norm...


[NeMo W 2022-03-29 12:57:34 nemo_logging:349] hifi-gan\stft.py:67: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
      fft_window = pad_center(fft_window, filter_length)
    


In [3]:
def infer(str_input):
    with torch.no_grad():
        parsed = spec_gen.parse(str_input)
        gen_spec_kwargs = {}
           
        spectrogram = spec_gen.generate_spectrogram(tokens=parsed, **gen_spec_kwargs)
        #audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)
        audio, audio_torch = vocoder.vocode(spectrogram)
#        audio = vocoder._bias_denoise(audio, spectrogram).squeeze(1)
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

#Результат

In [5]:
text_to_generate = input("Input text to synthesize: ")
print(text_to_generate)
spec, audio = infer(text_to_generate)
import IPython.display as ipd
import numpy as np

_, audio = infer(text_to_generate)
path_to_file = 'output.wav'
from scipy.io.wavfile import write
write(path_to_file, 22050, audio)

ipd.Audio(audio, rate=22050, autoplay=True)